QIIME2(설치 Virtual Box)  
https://docs.google.com/presentation/d/1P6udhJDnvC2eurthDtHubcg4e_yDTsbsUNnecMM_gU8/edit?usp=sharing  
https://docs.qiime2.org/2019.1/ (tutorial list)  
하기 내용은 상기 링크를 정리한 것이다.  

# QIIME2 Tutorial

## 설치 환경
* Ubuntu 16.04
* ANACONDA (가상환경, qiime2로 naming한다.) python 3.5.5(버전 3.6에서도 설치 확인)  
  
## QIIME2 tutorial list
* “Moving Pictures” tutorial
* Fecal microbiota transplant (FMT) study: an exercise
* “Atacama soil microbiome” tutorial
* Differential abundance analysis with gneiss
* Importing data
* Exporting data
* Metadata in QIIME 2
* Filtering data
* Training feature classifiers with q2-feature-classifier
* Evaluating and controlling data quality with q2-quality-control
* Predicting sample metadata values with q2-sample-classifier
* Performing longitudinal and paired sample comparisons with q2-longitudinal
* Identifying and filtering chimeric feature sequences with q2-vsearch
* Alternative methods of read-joining in QIIME 2
* Clustering sequences into OTUs using q2-vsearch


## “Moving Pictures” tutorial
https://docs.qiime2.org/2019.1/tutorials/moving-pictures/   
이번 분석은 5개의 시점을 두고 두 사람의 신체에서 추출한 신체 4곳의 미생물에 대해서 분석을 진행할 것이다.  
첫째로 antibiotic(항생물질)을 사용한다. 
  
## sample metadata
> ```$ mkdir Moving_Pictures```  
```$ cd Moving_Pictures```    
```(qiime2) $ wget -O "sample-metadata.tsv" "http://data.qiime2.org/2019.1/tutorials/moving-pictures/sample_metadata.tsv"```

### Obtaining and importing data  
  
> ```(qiime2) $ wget -O "emp-single-end-sequences/barcodes.fastq.gz" "https://data.qiime2.org/2019.1/tutorials/moving-pictures/emp-single-end-sequences/barcodes.fastq.gz"```  
```(qiime2) $ wget -O "emp-single-end-sequences/sequence.fastq.gz" "https://data.qiime2.org/2019.1/tutorials/moving-pictures/emp-single-end-sequences/sequences.fastq.gz"```

  
semantic type QIIME 2는 EMPSingleEndSequence이다. 
EMPSingleEndSequence artifacts는 아직 sequence가 sample에 지정되지 않은  multiplexed한 sequence에 담겨있다.(따라서, sequence.fastq.gz 및 barcodes.fastq.gz 파일을 포함한다. barcodes.fastq.gz에는 sequence.fastq.gz의 각 sequence에 barcode가 들어있다.)
>```(qiime2) $ qiime tools import --type EMPSingleEndSequences --input-path emp-single-end-sequences/ --output-path emp-single-end-sequences.qza```

### Demultiplexing sequences
demultiplex sequence는 각 sample 별로 관련된 barcode sequence를 아는 것이 필요하다. 
이 정보에는 sample metadata file이 담겨있다. 
command를 따라하면 sequences(demux emp-single command는 sequence에  barcode한 Earth Microbiome Project protocol 참조한다.)를 demultiplex할 수 있다.
demultiplexed sequences의 demux.qza를 QIIME2로 생성할 수 있다.  
  
> ```(qiime2) $ qiime demux emp-single \```  
```--i-seqs emp-single-end-sequences.qza \```  
```--m-barcodes-file sample-metadata.tsv \```  
```--m-barcodes-column BarcodeSequence \```  
```--o-per-sample-sequences demux.qza```
  
Demultiplexing 이후 생성된 파일을 이용하여 demultiplexing summary results를 생성한다.

> ```(qiime2) $ qiime demux summarize --i-data demux.qza --o-visualization demux.qzv```

* Visualization 

> ```(qiime2) $ qiime tools view demux.qzv```  
  
or https://view.qiime2.org/ 접속하여 파일을 드래그앤 드랍한다.  

예제) : https://view.qiime2.org/visualization/?type=html&src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Fdemux.qzv 



### Sequence quality control and feature table construction
  
QIIME 2 plugins은 몇가지(DADA2, Deblur, basic quality-score-based filtering) 질좋은 control method 이용할 수 있다.  
Tutorial에서는 DADA2와 Deblur를 사용하며 원하는 것으로 변경하면서 사용해도 무방하다.


#### Option 1: DADA2
DADA2는 Illumina amplicon sequence data를 detecting하고 correcting하는 pipeline이다.  
`q2-dada2` plugin을 통하여 사용이 가능하며, quality control process는 sequencing data에서 식별된 phiX판독 값을 추가적으로 filter한다.  
`*data2 denoise-single` method는 quality filtering에서 사용할 두가지 parameter를 요구한다.  
`--p-trim-left m`의 m에 시작하는 각 sequence 값을 넣고, `--p-trunc-len n`의 n에는 각 시퀀스를 자를 위치에 넣는다.   
위와 같이 pipeline을 통과하면 low quality sequences가 제거된다.



위와 같은 이유 때문에 m, n 값을 찾기 위해 이전단계에서 만들어진 `demux.qzv`파일에서 생성된 qiime demux summarize의 Interactive Quality Plot tab을 확인해야한다.  
`demux.qzv`의 quality plots을 살펴보면 초기 sequence 에서는 quality가 좋은 상황을 볼 수 있으면 sequence가 후반으로 갈수록 quality가 떨어지는 것을 확인 할 수 있다.  
때문에 넉넉히 120 position  drop off할 것이다.  
최대 10분까지 걸리며 모든 과정중에 제일 오래 걸린다.

> ```(qiime2) $ qiime dada2 denoise-single \ ```  
```--i-demultiplexed-seqs demux.qza \```   
```--p-trim-left 0 \```  
```--p-trunc-len 120 \```  
```--o-representative-sequences rep-seqs-data2.qza \```   
```--o-table table-data2.qza \```  
```--o-denoising-stats stats-data2.qza```

#### dada2 Output visualization
> ```(qiime2) $ qiime metadata tabulate \```  
```--m-input-file stats-data2.qza \```  
```--o-visualization stats-dada2.qzv```

dada2로 Tutorial을 지속 진행한다면, 아래와 같이 변환해준다.  
`$ mv rep-seqs-data2.qza rep-seqs.qza`   
`$ mv table-data2.qza table.qza`



#### Option 2 : Deblur

Deblur는 sequence error profile를 이용하여 읽은 erroneous sequence와 true biological sequence를 사용해 high quality sequence variant data를 생성한다.  
  
data 생성시에는 두 단계로 적용된다.
1. quality scores를 적용한 initial(처음의) quality filtering process를 기반으로 한다.  
Deblur는 다음의 [method](https://www.nature.com/articles/nmeth.2276)를 quality filtering approach로 이행한다.
> `(qiime2) $ qiime quality-filter q-score \`  
```--i-demux demux.qza \```  
```--o-filtered-sequences demux-filtered.qza \```  
```--o-filter-stats demux-filter-stats.qza```  
2. ```qiime deblur denoise-16S method``` 를 사용한다.  
다음 method는 ```-p-trim-length n```에서 사용할 parameter(quality filtering을 위한)가 필요하다.  
이때 `n`에는 축소한 sequence가 들어가야한다.  
일반적으로 Deblur를 사용하는 사용자들에게 추천하는 setting은 median quality scroe로 시작하고 drop이 적은 length를 넣는것이다.  
data에서 보듯이 115에서 130의 sequence position range를 제한한다.(QIIME 개발자의 주관적인 관점이므로 축소하거나 늘려도 상관없음)  
추천사항에서 예외적인 상황은 multiple sequencing을 실행하는 상황이다.  
위와 같은 meta-analysis에서 study-specific가 편향 적용되는 것으 피하기 위해 모두 같은 length로 읽는 것이 critical하다.

이미 qiime dada2 denosie-single에서 trim 120 lenght를 사용하였으므로, 120 lenght로 quality plots을 준다.  
> `(qiime2) $ qiime deblur denoise-16S \`  
`--i-demultiplexed-seqs demux-filtered.qza \`  
`--p-trim-length 120 \`  
`--o-representative-sequences rep-seqs-deblur.qza \`  
`--o-table table-deblur.qza \`  
`--p-sample-stats \`  
`--o-stats deblur-stats.qza`  
  
<이름변환>
>`$ mv rep-seqs-deblur.qza rep-seqs2.qza`  
`$ mv table-deblur.qza table2.qza`  

### FeatureTable and FeatureData summaries  
quality filtering단계가 끝난 후, 결과를 표시해야한다.  
아래 command를 따라서 visual summaries를 진행한다.   
Visual summaries는 각 sample과 관련된 많은 히스토그램, 요약 통계를 이용한 많은 정보를 제공할 것이다.   
feature-table tabulate-seqs 명령어는 각 feature Id의 sequense를 제공할것이며,   
또한 반대의 NCBI nt database 쉬운 sequence의 BLAST 도 제공할 것이다.(해석에 대한 수정필요..)  
다음의 visualization은 tutorial 중에서도 매우 유용하게 사용될 것이다.
> `(qiime2) $ qiime feature-table summarize `  
`--i-table table2.qza \`  
`--o-visualization table2.qzv \`  
`--m-sample-metadata-file sample-metadata.tsv`  
  
result link:  
https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Ftable.qzv  

> `(qiime2) $ qiime feature-table tabulate-seqs \`  
`--i-data rep-seqs2.qza \`  
`--o-visualization rep-seqs2.qzv`  

result link:  
https://view.qiime2.org/visualization/?type=html&src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Frep-seqs.qzv

### Generate a tree for phylogenetic diversity analyses  
QIIME은 Faith’s Phylogenetic Diversity와 weighted 그리고 unweighted UniFrac를 포함한 phylogenetic diversity metric에 대한 몇가지를 지원한다.  
샘플당 feature 수 이외에, 측정기준 feature들에 대한 관계를 형성하는 rooted phylogenetic tree의 metrics가 필요하다.  
정보는 `Phylogeny[Rooted]`에 저장된다.

q2-phylogeny plugin으로 부터 나온 `align-to-tree-mafft-fastree`의 pipeline을 이용하여 plugin을 생성한다.  
첫번째로 pipeline은 QIIME artifact 부터 생성한 multiple sequence로 설정된 mafft program을 사용한다.  
다음은 pipeline filter(or masks)을 이용하여 highly variable 위치를 제거한다.  
이후 position들의 phylogenetic tree 부터 생성된 noise을 일반적으로 고려한다.(수정 필요..)  
> `(qiime2) $ qiime phylogeny align-to-tree-mafft-fasttree \`  
`--i-sequences rep-seqs2.qza \`  
`--o-alignment aligned-rep-seqs.qza \`  
`--o-masked-alignment masked-aligned-rep-seqs.qza \`  
`--o-tree unrooted-tree.qza `



### Alpha and beta diversity analysis
QIIME2는 `q2-diversity plugin`를 이용하여 diversity analysis를 진행할 수 있다.   
plugin은 alpha와 beta diversity metric를 계산을 지원하며 통계적으로 test하는 것도 지원하고 visualization과 상호작용 하는 것도 지원한다.  
처음으로는 `core-metrics-phylogenetic` method를 이용해야한다.  
Alpha와 Beta diversity의 analysis방법의 default는 다음과 같다.  
* Shannon’s diversity index (a quantitative measure of community richness)
* Observed OTUs (a qualitative measure of community richness)
* Faith’s Phylogenetic Diversity (a qualitative measure of community richness that incorporates phylogenetic relationships between the features)
* Evenness (or Pielou’s Evenness; a measure of community evenness)
* Jaccard distance (a qualitative measure of community dissimilarity)
* Bray-Curtis distance (a quantitative measure of community dissimilarity)
* unweighted UniFrac distance (a qualitative measure of community dissimilarity that incorporates phylogenetic relationships between the features)
* weighted UniFrac distance (a quantitative measure of community dissimilarity that incorporates phylogenetic relationships between the features)  
  
중요한 parameter는 --p-sampling-depth 를 이용하는 것이 필요하다.  
각 sampling 짝수로 depth에 적용해야한다.  
왜냐하면 대부분의 diversity metrics는 다른 sample에 다른 sampling depth에 민감하기 때문이다.  
이번 script는 각 제공된 sample count를 매개 변수로 제공된 값으로 임의적으로 subsample한다.  
> `(qiime2) $ qiime diversity core-metrics-phylogenetic \`  
`--i-phylogeny rooted-tree.qza \`  
`--i-table table.qza \`  
`--p-sampling-depth 1109 \`  
`--m-metadata-file sample-metadata.tsv \`  
`--output-dir core-metrics-results`  
  
#### Output visualization
* core-metrics-results/unweighted_unifrac_emperor.qzv :
    * [link](https://view.qiime2.org/visualization/?type=html&src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Funweighted_unifrac_emperor.qzv)
* core-metrics-results/jaccard_emperor.qzv :
    * [link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Fjaccard_emperor.qzv)
* core-metrics-results/bray_curtis_emperor.qzv : 
    * [link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Fbray_curtis_emperor.qzv)
* core-metrics-results/weighted_unifrac_emperor.qzv : 
    * [link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Fweighted_unifrac_emperor.qzv)


Diversity metrics를 계산 후 샘플들을의 microbial composition 시작해야한다.  
sample metadata의 대한 파일은 [링크](https://docs.google.com/spreadsheets/d/1yFL_Mwh7b5LE5vW06KooZFB_cjoEJEXGFwhos6X7jns/edit#gid=0)와 같다.  
* alpha-diversity(faith-pd-group-significance)
> `(qiime2) $ qiime diversity alpha-group-significance \`  
`--i-alpha-diversity core-metrics-results/faith_pd_vector.qza \`  
`--m-metadata-file sample-metadata.tsv \`   
`--o-visualization core-metrics-results/faith-pd-group-significance.qzv`
* alpha-diversity(evenness-pd-group-significance)
> `(qiime2) $ qiime diversity alpha-group-significance \`  
`--i-alpha-diversity core-metrics-results/evenness_vector.qza \`  
`--m-metadata-file sample-metadata.tsv \`  
`--o-visualization core-metrics-results/evenness-group-significance.qzv`  
  
  
* core-metrics-results/faith-pd-group-significance.qzv:
    * [link](https://view.qiime2.org/visualization/?type=html&src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Ffaith-pd-group-significance.qzv)
* core-metrics-results/evenness-group-significance.qzv:
    * [link](https://view.qiime2.org/visualization/?type=html&src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Fevenness-group-significance.qzv)

다음은 beta-diversity에 대한 코드이다.  
input distance로 사용할 matrix는 unweighted UniFrac distances이며 두개의 샘플코드로 이루어졌다.


* beta-diversity(unweighted-unifrac-body-site-significance)
> `(qiime2) $ qiime diversity beta-group-significance \`  
  `--i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza \`  
  `--m-metadata-file sample-metadata.tsv \`  
  `--m-metadata-column BodySite \`  
  `--o-visualization core-metrics-results/unweighted-unifrac-body-site-significance.qzv \`  
  `--p-pairwise`  
* beta-diversity(unweighted-unifrac-subject-group-significance)
> `(qiime2) $ qiime diversity beta-group-significance \`  
  `--i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza \`  
  `--m-metadata-file sample-metadata.tsv \`  
  `--m-metadata-column Subject \`  
  `--o-visualization core-metrics-results/unweighted-unifrac-subject-group-significance.qzv \`  
  `--p-pairwise`
* core-metrics-results/unweighted-unifrac-body-site-significance.qzv:
    * [link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Funweighted-unifrac-body-site-significance.qzv)
* core-metrics-results/unweighted-unifrac-subject-group-significance.qzv:
    * [link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Funweighted-unifrac-subject-group-significance.qzv)  
  
Again, none of the continuous sample metadata that we have for this data set are correlated with sample composition, so we won’t test for those associations here. If you’re interested in performing those tests, you can use the qiime metadata distance-matrix in combination with qiime diversity mantel and qiime diversity bioenv commands.

### Emperor
마지막은 microbial community composition approach에 대한 것이다.  
Emperor tool을 이용하여 PCoA(principal coordinates) plot을 탐색 할 수 있다.  
  
While our core-metrics-phylogenetic command did already generate some Emperor plots, we want to pass an optional parameter, --p-custom-axes, which is very useful for exploring time series data. The PCoA results that were used in core-metrics-phylogeny are also available, making it easy to generate new visualizations with Emperor. We will generate Emperor plots for unweighted UniFrac and Bray-Curtis so that the resulting plot will contain axes for principal coordinate 1, principal coordinate 2, and days since the experiment start. We will use that last axis to explore how these samples changed over time.  
> `(qiime2) $ qiime emperor plot \`  
  `--i-pcoa core-metrics-results/unweighted_unifrac_pcoa_results.qza \`  
  `--m-metadata-file sample-metadata.tsv \`  
  `--p-custom-axes DaysSinceExperimentStart \`  
  `--o-visualization core-metrics-results/unweighted-unifrac-emperor-DaysSinceExperimentStart.qzv`  
  
  
> `(qiime2) qiime emperor plot \`  
  `--i-pcoa core-metrics-results/bray_curtis_pcoa_results.qza \`  
  `--m-metadata-file sample-metadata.tsv \`  
  `--p-custom-axes DaysSinceExperimentStart \`  
  `--o-visualization core-metrics-results/bray-curtis-emperor-DaysSinceExperimentStart.qzv`  
  
* core-metrics-results/unweighted-unifrac-emperor-DaysSinceExperimentStart.qzv:
    * [link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Funweighted-unifrac-emperor-DaysSinceExperimentStart.qzv)
* core-metrics-results/bray-curtis-emperor-DaysSinceExperimentStart.qzv:
    * [link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Fbray-curtis-emperor-DaysSinceExperimentStart.qzv)
    
### Alpha rarefaction plotting
QIIME의 alpha-diversity를 이용하여 rarefaction visualizer를 해보도록 한다.
> `(qiime2) $ qiime diversity alpha-rarefaction \`  
`--i-table table.qza \`  
`--i-phylogeny rooted-tree.qza \`  
`--p-max-depth 4000 \`  
`--m-metadata-file sample-metadata.tsv \`  
`--o-visualization alpha-rarefaction.qzv`  
  
* alpha-rarefaction.qzv:
    * [link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Falpha-rarefaction.qzv)

### Taxonomic analysis
다음은 taxonomic composition 에 대한 섹션이다.  
처음에는 QIIME에 taxonomy sequences를 지정하는 것이다.  
이때 pre-trained Naive Bayes classifier와 q2-feature-classifier plugin을 사용할 것이다.  
classifier 학습은 Greengenes 13_8 99% OTUs로 할 것이다.  
> `$ wget \`  
  `-O "gg-13-8-99-515-806-nb-classifier.qza" \`  
  `"https://data.qiime2.org/2019.1/common/gg-13-8-99-515-806-nb-classifier.qza"`  
`$ pip install scikit-learn==0.20.2`

> `(qiime2) $ qiime feature-classifier classify-sklearn \`  
  `--i-classifier gg-13-8-99-515-806-nb-classifier.qza \`  
  `--i-reads rep-seqs.qza \`  
  `--o-classification taxonomy.qza`  
  
> `(qiime2) $ qiime metadata tabulate \`  
  `--m-input-file taxonomy.qza \`  
  `--o-visualization taxonomy.qzv`
  
* taxonomy.qzv:
    * [link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Ftaxonomy.qzv)  
    
Next, we can view the taxonomic composition of our samples with interactive bar plots. Generate those plots with the following command and then open the visualization.  
> `(qiime2) $ qiime taxa barplot \`  
  `--i-table table.qza \`  
  `--i-taxonomy taxonomy.qza \`  
  `--m-metadata-file sample-metadata.tsv \`  
  `--o-visualization taxa-bar-plots.qzv`  
  
* taxa-bar-plots.qzv:
    * [link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Ftaxa-bar-plots.qzv)  

----

## Fecal microbiota tansplant(FMT) study: an exercise

Tutorial을 위하여 data는 [Fecal Microbiome Transplant study](https://microbiomejournal.biomedcentral.com/articles/10.1186/s40168-016-0225-7)에서 받아왔다.  
FMT는 autism(자폐증)과 gastrointestinal disorders(위장장애)를 가진 18세 이하 아이들의 변을 Autism Diagnostic Interview-Revised(ADI-R: 위장 증상 평가 척도)및,   
Gastrointestinal Symptom Rating Scale(GSRS)로 microbiota의 움직임과 증상으로 위장 증상을 완화하기 위해 측정한 데이터이다.  
  
The Parent Global Impressions-III(PGI-III)과 Childhood Autism Rating Scale(CARS)를 포함한 위장 심각도에 대한 몇가지 측정 기준으로 위장 미생물의 18주 동안 변화를 통해 심각도를 점수화한다.  
microbiome은 배설물 면봉 sample(화장실에서 사용된 휴지로 부터 수집됨)과 적은 횟수의 배설물 sample을 한주 동안 모은 것으로 추적되었다.  
  
전체 연구에서, 실험의 안정성을 위해 18명 환자는 치료를 받았고 20명의 환자는 대조군으로 남았다.  
대조군은 치료 받지 않았으나 gut microbiom의 변화를 확인하기위해  주기적을로 monitor하였다.  
연구에서 치료 하는동안에는 이식된 The fecal material sequenced 분석되었다.  
  
The tutorial dataset은 data의 subsample로 이번 학습에서 생성되었다.  
각 기 다른 5개의 치료 방법으로 치료된 5명의 데이터가 포함되어 있다.  
6, 16은 각 사람의 stool과 fecal swab가 포함된 sample이다.  
이 sample은 FMT치료 이전과 이후의 sample이다.  
5개 sample은 fecal에 transplant한 측정값을 포함하고 있다.  
  
Moving Pictures tutorial과 같이 DADA2를 사용하여 initial quality control을 진행하였고 FeatureTable과 FeatureData object를 생성하였다.   
그러나 DADA2 denoising을 진행할때 single sequencing만 진행하였다.  
때문에 각 sequencing을 실행할때 기본적으로 다음의 결과를 병합해야한다.  
  
### Obtain data file
> `(qiime2) $ mkdir fmt-tutorial`  
`(qiime2) $ cd fmt-tutorial/`  
`(qiime2) $ wget \`  
  `-O "sample-metadata.tsv" \`  
  `"https://data.qiime2.org/2019.1/tutorials/fmt/sample_metadata.tsv"`  
  
다음으로 필요한 것은 demultiplexed analysis를 진행할 sequences데이터를 받아야 한다.  
두개의 demultiplexed sequences set을 받아야하며, 각 sequence를 동일하게 실행해야 한다.  

이번 tutorial에서는 complete sequence data의 small subsample이용하여 빠르게 실행 할 수 있다.  
현재 진행하는 test는 10% subsample data를 이용한다.  
  
> `(qiime2) $ wget -O "fmt-tutorial-demux-1.qza" "https://data.qiime2.org/2019.1/tutorials/fmt/fmt-tutorial-demux-1-10p.qza"`  
`(qiime2) $ wget -O "fmt-tutorial-demux-2.qza" "https://data.qiime2.org/2019.1/tutorials/fmt/fmt-tutorial-demux-2-10p.qza"`

### Sequence Quality control
quality control을 위하여 demultiplexed sequences를 DADA2에 사용할 것이다.  
**그러나**   
demultiplexed sequnces에 개별적으로 denoise-single명령어를 실행한다.   
> `(qiime2) $ qiime demux summarize \`  
`--i-data fmt-tutorial-demux-1.qza \`  
`--o-visualization demux-summary-1.qzv`  
  
* demux-summary-1.qzv:  
[link](https://view.qiime2.org/visualization/?type=html&src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Ffmt%2Fdemux-summary-1.qzv)  

> `(qiime2) $ qiime demux summarize \`  
`--i-data fmt-tutorial-demux-2.qza \`  
`--o-visualization demux-summary-2.qzv`  
  
* demux-summary-2.qzv:  
[link](https://view.qiime2.org/visualization/?type=html&src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Ffmt%2Fdemux-summary-2.qzv)  
  
Here the quality seems relatively low in the first few bases, and then seems to stay relatively high through the end of the reads.   
We’ll therefore trim the first 13 bases from each sequence and truncate the sequences at 150 bases.  
Since the reads are 151 bases long, this results in very little truncation of the sequences.
  
> `(qiime2) $ qiime dada2 denoise-single \`  
`--p-trim-left 13 \`  
`--p-trunc-len 150 \`  
`--i-demultiplexed-seqs fmt-tutorial-demux-1.qza \`  
`--o-representative-sequences rep-seqs-1.qza \`  
`--o-table table-1.qza \`  
`--o-denoising-stats stats-1.qza`  
  
> `(qiime2) $ qiime dada2 denoise-single \`  
`--p-trim-left 13 \`  
`--p-trunc-len 150 \`  
`--i-demultiplexed-seqs fmt-tutorial-demux-2.qza \`  
`--o-representative-sequences rep-seqs-2.qza \`  
`--o-table table-2.qza \`  
`--o-denoising-stats stats-2.qza`  

### Viewing denoising stats
기본적인 denoising process 통계이다.
> `(qiime2) $ qiime metadata tabulate \`  
`--m-input-file stats-1.qza \`  
`--o-visualization denoising-stats-1.qzv`  
  
* denoising-stats-1.qzv:  
[link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Ffmt%2Fdenoising-stats-1.qzv)

> `(qiime2) $ qiime metadata tabulate \`  
`--m-input-file stats-2.qza \`  
`--o-visualization denoising-stats-2.qzv`  
  
* denoising-stats-2.qzv:   
[link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Ffmt%2Fdenoising-stats-2.qzv)




### Merging denoised data  
Merging denoised data step은 각 sequencing 을 실행하는 마지막 단계이다.  
denosined된 두개의 sequence를 합치는 단계로서 table1과 table2를 합치면된다.  
(in this case, the feature id is the md5 hash of the sequence defining the feature).  
  
table merge
> `(qiime2) $ qiime feature-table merge \`  
`--i-tables table-1.qza \`  
`--i-tables table-2.qza \`  
`--o-merged-table table.qza `
  
representative sequence merge  
> `(qiime2) $ qiime feature-table merge-seqs \`  
`--i-data rep-seqs-1.qza \`  
`--i-data rep-seqs-2.qza \`  
`--o-merged-data rep-seqs.qza`  

### Diversity analysis
FeatureTable, FeatureData객체가 생성되었으므로 metadata로 구성된 sample을 분석 할 준비가 되었다.
`the moving pictures`에서 사용된 코드를 이용한다.
다음에서 나오는 질문들은 longitudinal 에 대한 분석이다. longitudinal은 q2-longituinal tutorial에서 확인할 수 있다.
각 질문에 대한 결과를 수집하는게 목적이다.
  
https://docs.qiime2.org/2019.1/tutorials/fmt/ 질문에 대한 링크 (추후에 진행)

----

## "Atacama soil microbiome" tutorial  
튜토리얼은 두개의 목적이 있다.  
첫번째 목적은 initial processing의 paired-end를 읽는것을 분석하는 것이고,  
동일한 시점 까지 single-end 설명의 읽고 분석을 하여 설명하는 것이다.  
이번 단계에서는 importing, demultiplexing, and denoising step그리고 feature table과 feature와 관련된 결과도 포함되어있다.  
  
The Atacama Desert는 지구상에서 가장 건조한 지역이며, per decade 동안 비가 1millimeter 이하로 비가 내린다.  
그럼에도 불구하고 극단적으로 건조한 상황의 흙속에서 microbiome이 살고 있다.  
이번 연구에서 profiled된 soil microbiome은 높이와 습도가 양의 상관관계에 있는 두 동서 횡단인 Baquedano와 Yungay를 따른다.  
횡단을 따라 각 지역에서 토양샘플을 추출하였고 각기 다른 구덩이의 3가지 깊이에서 수집하였다.  
> `(qiime2) $ mkdir qiime2-atacama-tutorial`  
`(qiime2) $ cd qiime2-atacama-tutorial/`  
`(qiime2) $ wget -O "sample-metadata.tsv" "https://data.qiime2.org/2019.1/tutorials/atacama-soils/sample_metadata.tsv"`  


### 10% subsample data
다음은 multiplexed를 읽는 것이다. forward, reverse, 와 barcode fastq.gz파일을 다운로드해야 한다.
> `$ mkdir emp-paired-end-sequences`  
  
* forword  
  
> `(qiime2) $ wget -O "emp-paried-end-sequences/forward.fastq.gz" \`  
`"https://data.qiime2.org/2019.1/tutorials/atacama-soils/10p/forward.fastq.gz"`
  
* reverse  
  
> `(qiime2) $ wget -O "emp-paried-end-sequences/reverse.fastq.gz" \`  
`"https://data.qiime2.org/2019.1/tutorials/atacama-soils/10p/reverse.fastq.gz"`  
  
* barcodes  
  
> `(qiime2) $ wget -O "emp-paried-end-sequences/barcodes.fastq.gz" \`  
`"https://data.qiime2.org/2019.1/tutorials/atacama-soils/10p/barcodes.fastq.gz"`  
  
### Paired-end read analysis commands

data를 분석하려면 이전에 다운받았던 3가지 `EMPPairedEndSequences` 파일의 유형을 바꿔줘야한다.  
* import  
  
> `(qiime2) $ qiime tools import \`  
`--type EMPPairedEndSequences \`  
`--input-path emp-paried-end-sequences \`  
`--output-path emp-paired-end-sequences.qza`  



### Read demultiplex sequence  
다음은 demultiplex sequence를 읽는 것이다.  
sequence를 읽기 위해서는 metadata file sample을 요구한다.  
그리고 해당 파일에서 per-sample barcode가 포함된 column을 지정해야한다.  
 In this data set, the barcode reads are the reverse complement of those included in the sample metadata file,  
 so we additionally include the --p-rev-comp-mapping-barcodes parameter.  
 #### Barcode mapping
> `(qiime2) $ qiime demux emp-paired \`  
`--m-barcodes-file sample-metadata.tsv \`  
`--m-barcodes-column BarcodeSequence \`  
`--i-seqs emp-paired-end-sequences.qza \`  
`--o-per-sample-sequences demux.qza \`  
`--p-rev-comp-mapping-barcodes`


#### Demultiplex summarize
> `(qiime2) $ qiime demux summarize \`  
`--i-data demux.qza \`  
`--o-visualization demux.qzv`  
  
* demux.qzv:  
    * [link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fatacama-soils%2Fdemux.qzv)

### Quality sequence
demultiplexing 이후,  무작위로 선발된 sample을 기반으로 sequence quality를 살펴보고 denoise를 진행해야 한다.  
quality plot을 볼때, “moving pictures tutorial”과는 다르게 두개의 plots의 sample을 가지고 있는 것을 주의해야한다.  
plot left에는 forward로 읽은 quality score를 표시하고, plot right에는 reverse로 읽은 quality score를 표시한다.  
우리는 plot을 사용하여 trimming parameters를 결정하고 dada2 denoise-paired를 이용하여 denoise를 진행한다.   
이번 예제에서는 150의 forward와 reverse로 읽는다.  
Since we need the reads to be long enough to overlap   
paired의 처음과 끝이 연결될때 충분히 길어야 하기 때문에 13개의 forward와 reverse로 trimmed된다,  
그러나 판독 길이가 너무 작아지지 않도록 sequence의 끝 부분에 trimming을 적용하지 않는다.  
> `(qiime2) $ qiime dada2 denoise-paired \`  
`--i-demultiplexed-seqs demux.qza \`  
`--p-trim-left-f 13 \`  
`--p-trim-left-r 13 \`  
`--p-trunc-len-f 150 \`  
`--p-trunc-len-r 150 \`  
`--o-table table.qza \`  
`--o-representative-sequences rep-seqs.qza \`  
`--o-denoising-stats denoising-stats.qza`  


In [ ]:
qiime dada2 denoise-paired \
--i-demultiplexed-seqs data_1.qza \
--p-trim-left-f 13 \
--p-trim-left-r 13 \
--p-trunc-len-f 300 \
--p-trunc-len-r 300 \
--o-table table.qza \
--o-representative-sequences test_rep-seqs.qza \
--o-denoising-stats test_denoising-stats.qza

$$
\overrightarrow{y}\;=\;\overrightarrow{\beta_0}\;+\;\overrightarrow{\beta_{subject}}\;\overrightarrow{X_{subject}}\;+\;\overrightarrow{\beta_{sex}}\;\overrightarrow{X_{sex}}\;+\;\overrightarrow{\beta_{age}}\;\overrightarrow{X_{age}}\;+\; \overrightarrow{\beta_{sCD14ugml}}\;\overrightarrow{X_{sCD14ugml}}\;+\;\overrightarrow{\beta_{LBPugml}}\;\overrightarrow{X_{LBPugml}} 
$$

$$
b_i = \sqrt{\frac{rs}{r+s}} log\frac{g(x_r)}{g(x_s)}
$$

$$
d(x, y) \; = \; V[ln \frac{x}{y}]
$$

----

## Performing longitudinal and paired sample comparisons with q2-longitudinal  
  
q2-longitudinal의 various features에 대해 시연할 것이다.  
이 plugin은 통계 longitudinal 학습 설계의 visual comparisons과 paired sample을 지원하며,  
이런 것들로 인해 ‘stats’간에 if/how sample이 변경되는지에 대해 결정하게 된다.  
‘stats’는 일반적으로 시간과 environmental gradient과 관련이 있고,  
paired analyses의 경우 서로 다른 시간에서 관찰된 두 시점에서 관찰된 같은 대상으로 구성되어야 한다.  
예를 들어 임상 환자의 대변샘플의 치료받기 전과 치료 후의 샘플을 말하는 것이다.  
  
“States”는 일반적으로 methodological으로 사용할 수 있으며,  
이 sample 두시 점에 대한 같은 sample을 말한다.  
예를 들어, q2-longitudinal는 여러가지 수집방법, 저장 방법, DNA 추출 방법 또는 각 sample의 feature 끼리 비교에 관한 bioinformatic processing step에 비교하는 것이 가능하다.
  
이번 예제에서 사용할 데이터는 ECAM study데이터 이다.  
연구 내용은 2세까지의 영아와 엄마의 microbiota에 대한 longitudinal에 대한 연구이다.  
  
### Obtain data
> `(qiime2) $ mkdir longitudinal-tutorial`  
`(qiime2) $ cd longitudinal-tutorial`  
`(qiime2) $ wget -O "ecam-sample-metadata.tsv" "https://data.qiime2.org/2019.1/tutorials/longitudinal/sample_metadata.tsv"`  
`(qiime2) $ wget   -O "shannon.qza"   "https://data.qiime2.org/2019.1/tutorials/longitudinal/ecam_shannon.qza"`  
`(qiime2) $ wget   -O "unweighted_unifrac_distance_matrix.qza"   "https://data.qiime2.org/2019.1/tutorials/longitudinal/unweighted_unifrac_distance_matrix.qza"`  
  
### Pairwise difference comparisons
  
  Pairwise difference test는 특정 측정 항목의 값이 치료 전후간에 변하는지 여부를 결정한다.  
이번 visualizer는 현재 feature table안의 또는 sample metadata file안의 metadata values에 대한 feature abundance간의 비교를 지원한다.  
Alpha diversity values(e.g., observed sequence variants)와 Beta diversity values(e.g., principal coordinates)가 test를 할 metrics를 비교하는 것에 유용하다.  
그리고 input으로 하나의 metadata가 담겨있는 것을 넣어야 한다.  
> `(qiime2) $ qiime longitudinal pairwise-differences \`  
`--m-metadata-file ecam-sample-metadata.tsv \`  
`--m-metadata-file shannon.qza \`  
`--p-metric shannon \`  
`--p-group-column delivery \`  
`--p-state-column month \`  
`--p-state-1 0 \`  
`--p-state-2 12 \`  
`--p-individual-id-column studyid \`  
`--p-replicate-handling random \`  
`--o-visualization pairwise-differences.qzv `  
  
* pairwise-differences.qzv:  
    * [link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Flongitudinal%2Fpairwise-differences.qzv)

pairwise-distances visualizer 또한 두개의 각각 다른 “states”가 바뀐 paired sample에 대해 평가하나,  
metadata column 또는 artifact를 input으로 가져오는 대신에 “pre”와 “post” sample 쌍에 대해 거리를 평가하기 위해 거리 matrix를 사용하고 group-column매개 변수에 지정된대로 서로 다른 paired에 차이가 큰지에 대한 여부를 테스트한다.  
> `(qiime2) $ qiime longitudinal pairwise-distances \`  
`--i-distance-matrix unweighted_unifrac_distance_matrix.qza \`  
`--m-metadata-file ecam-sample-metadata.tsv \`   
`--p-group-column delivery \`  
`--p-state-column month \`  
`--p-state-1 0 \`  
`--p-state-2 12 \`  
`--p-individual-id-column studyid \`  
`--p-replicate-handling random \`  
`--o-visualization pairwise-distances.qzv`  
  
* pairwise-distances.qzv: view 

### Linear mixed effect models
Linear mixed effects(LME) models single response variable과 하나 또는 다수의 independent variable관계에 대해 test한다.  
예를 들어 repeated-measures sampleling 실험에서 dependent sample간의 관찰이 이루어진다.  
이번 그래프는 LME model을 이용하여 회귀 그래프를 그린다.  
추가적으로 반복되는 데이터를 구별하기 위해 individual-id-column매개 변수로 열을 지정해야한다.  
위 매개변수로 table의 ID기능을 수행한다.  
  
쉼표로 구분하여 임의대로 column을 입력할 수 도 있다.  
여기서는 LME를 사용하여 alpha diversity(Shannon diversity index)가 ECAM dataset의 delivery, mode, diet, 그리고 sex 따라 변동 여부를 test한다.  

> `(qiime2) $ qiime longitudinal linear-mixed-effects \`  
`--m-metadata-file ecam-sample-metadata.tsv \`  
`--m-metadata-file shannon.qza \`  
`--p-metric shannon \`  
`--p-group-columns delivery,diet,sex \`  
`--p-state-column month \`  
`--p-individual-id-column studyid \`  
`--o-visualization linear-mixed-effects.qzv \`  
   
* linear-mixed-effects.qzv: 
    * [link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Flongitudinal%2Flinear-mixed-effects.qzv)  
  
Visualizer produced는 몇가지 명령어를 담고 있다.  
첫째 input parameters를 보여주는 편리성을 가진 visualization
둘째 학습된 LEM model의 model summary 이 정보는 각 "groups"에 대해 설명한다.
조사해야될 주요 결과는 "model results"와 "model summary"이다.
  
뒤의 내용은 추후에 정리

### Volatility analysis
Volatility visualizer는 dependent variable이 얼마나 continuous한지,  
independent variable가 얼마나 volatile한지 서로 상호 작용하는 line plot을 생성한다.  
Multiple metadata file과 FeatureTable[Relativefrequency] table은 넣은 것으로 interactive visualization에 사용하는 것이 가능하며,  
또한 다른 dependent variables를 y-axis에 대한 plot으로 선택할 수 있다.
  
> `(qiime2) $ qiime longitudinal volatility \`  
`--m-metadata-file ecam-sample-metadata.tsv \`  
`--m-metadata-file shannon.qza \`  
`--p-default-metric shannon \`  
`--p-default-group-column delivery \`  
`--p-state-column month \`  
`--p-individual-id-column studyid \`  
`--o-visualization volatility.qzv`  
  
* volatility.qzv: 
    * [link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Flongitudinal%2Fvolatility.qzv)  
 
용어 설명:
* cesarean: 자궁
* vagina: 질
  
Volatility대한 자세한 정리는 추후에 정리

### First diferencing to track rate of change  
Time series data를 보는 다른 방법은 시간의 변화에 따른 change 비율이다.  
  
$$
\Delta Y_t = Y_t -Y_{t-1}
$$
  
Y는 metric의 값이고 t가 시간일 때, $\Delta Y_t$값은 $Y_t - Y_{t-1}$으로 계산이 가능하다.  
계산은 고정된 간격으로 시행되므로 $Y_t$나 $Y_{t-1}$이 없을 경우에는 계산되지 않는다.  
> `(qiime2) $ qiime longitudinal first-differences \`  
`--m-metadata-file ecam-sample-metadata.tsv \`   
`--m-metadata-file shannon.qza \`  
`--p-state-column month \`   
`--p-metric shannon \`   
`--p-individual-id-column studyid \`    
`--p-replicate-handling random \`  
`--o-first-differences shannon-first-differences.qza`  
    
출력값은 SampleData[firstDifferences]이며, 시각화 된다.
예를 들어 volatility visualizer하거나 linear-mixed-effects 또는 other method로 분석할 수 있다.
